In [18]:
import datetime
import json
import os
from pathlib import Path
from pprint import pprint
import time
from zipfile import ZipFile

from planet import api
from planet.api import filters
import rasterio
from rasterio import plot

import json
import os

#### Step 1: Initialize API client

In [19]:
# if your Planet API Key is not set as an environment variable, you can paste it below
API_KEY = os.environ.get('PL_API_KEY', 'PASTE_YOUR_KEY_HERE')

client = api.ClientV1(api_key=API_KEY)

In [20]:
# iowa crops aoi
test_aoi_geom = {
    "type": "Polygon",
    "coordinates": [
        [
            [-93.299129, 42.699599],
            [-93.299674, 42.812757],
            [-93.288436, 42.861921],
            [-93.265332, 42.924817],
            [-92.993873, 42.925124],
            [-92.993888, 42.773637],
            [-92.998396, 42.754529],
            [-93.019154, 42.699988],
            [-93.299129, 42.699599]
        ]
    ]
}

In [21]:
ids = ['20190415_170304_85_1068']

In [22]:
# specify the psscene4band surface reflectance product
# note: capitalization really matters in item_type when using planet client orders api
item_type = 'PSScene4Band'
bundle = 'analytic_sr_udm2'

##### Step 3.1: Build Orders Toolchain

In [23]:
# specify tools

# clip to AOI
clip_tool = {'clip': {'aoi': test_aoi_geom}}

# convert to NDVI
bandmath_tool = {'bandmath': {
    "pixel_type": "32R",
    "b1": "(b4 - b3) / (b4+b3)"
}}

tools = [clip_tool, bandmath_tool]
pprint(tools)

[{'clip': {'aoi': {'coordinates': [[[-93.299129, 42.699599],
                                    [-93.299674, 42.812757],
                                    [-93.288436, 42.861921],
                                    [-93.265332, 42.924817],
                                    [-92.993873, 42.925124],
                                    [-92.993888, 42.773637],
                                    [-92.998396, 42.754529],
                                    [-93.019154, 42.699988],
                                    [-93.299129, 42.699599]]],
                   'type': 'Polygon'}}},
 {'bandmath': {'b1': '(b4 - b3) / (b4+b3)', 'pixel_type': '32R'}}]


In [24]:
# specify a name
name = 'tutorial_order'

##### submit order

In [25]:
# zip up entire order into one file
ziptype = 'order'

# format the ids for use with the CLI
cli_ids = ','.join(ids)

# save tools definition to file
tools_file = 'tools.json'
with open(tools_file, 'w') as dst:
    dst.write(json.dumps(tools))
    
order_info_file = 'order.json'

In [26]:
# submit the order and save the response to a file so we can get the order id
!planet orders create \
    --id $cli_ids \
    --item-type $item_type \
    --bundle $bundle \
    --zip $ziptype \
    --tools $tools_file \
    --name $name | tee $order_info_file

{"_links": {"_self": "https://api.planet.com/compute/ops/orders/v2/9084691e-93ad-4eb0-9746-8ac14ab0e54f"}, "created_on": "2020-06-23T20:28:53.405Z", "delivery": {"archive_filename": "{{name}}_{{order_id}}.zip", "archive_type": "zip", "single_archive": true}, "error_hints": [], "id": "9084691e-93ad-4eb0-9746-8ac14ab0e54f", "last_message": "Preparing order", "last_modified": "2020-06-23T20:28:53.405Z", "name": "tutorial_order", "notifications": {}, "products": [{"item_ids": ["20190415_170304_85_1068"], "item_type": "PSScene4Band", "product_bundle": "analytic_sr_udm2"}], "state": "queued", "tools": [{"clip": {"aoi": {"coordinates": [[[-93.299129, 42.699599], [-93.299674, 42.812757], [-93.288436, 42.861921], [-93.265332, 42.924817], [-92.993873, 42.925124], [-92.993888, 42.773637], [-92.998396, 42.754529], [-93.019154, 42.699988], [-93.299129, 42.699599]]], "type": "Polygon"}}}, {"bandmath": {"b1": "(b4 - b3) / (b4+b3)", "pixel_type": "32R"}}]}


In [31]:
# read the order id
with open(order_info_file, 'r') as src:
    order_info = json.load(src)
    
order_id = order_info['id']
order_id

'9084691e-93ad-4eb0-9746-8ac14ab0e54f'

In [30]:
def poll_for_success(order_id, client, num_loops=50):
    count = 0
    while(count < num_loops):
        count += 1
        order_info = client.get_individual_order(order_id).get()
        state = order_info['state']
        print(state)
        success_states = ['success', 'partial']
        if state == 'failed':
            raise Exception(response)
        elif state in success_states:
            break
        
        time.sleep(10)
        
poll_for_success(order_id, client)

running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
success


In [32]:
demo_data_dir = os.path.join('data', 'demo')

# make the download directory if it doesn't exist
Path(demo_data_dir).mkdir(parents=True, exist_ok=True)

In [33]:
!planet orders download --dest $demo_data_dir $order_id

activating: 0            complete: 0              downloaded: 0.0MB             
downloading: 0           elapsed: 0               paging: False                 
pending: 0                                                                      

activating: 0            complete: 2              downloaded: 0.00MB            
downloading: 0           elapsed: 1               paging: False                 
pending: 0                                                                      

activating: 0            complete: 0              downloaded: 0.0MB             
downloading: 0           elapsed: 2               paging: False                 
pending: 0                                                                      

activating: 0            complete: 0              downloaded: 0.0MB             
downloading: 0           elapsed: 2               paging: False                 
pending: 0                                                                      



##### Step 4.3: Get Downloaded File Location(s)

We use the downloaded order manifest to find the downloaded file locations. The manifest is saved in the download directory.

In [40]:
!ls data/demo

tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f.zip


In [50]:
location = 'data/demo/tutorial_order_{}.zip'.format(order_id)
location

'data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f.zip'

#### Step 5: Unzip and View Order

In this step we will simply unzip the order and view the downloaded images and their usable data masks.

##### 5.1: Unzip Order

We will unzip the order into a directory named after the file, then we will find the downloaded files (they are in a `files` subdirectory)

In [42]:
def unzip(filename):
    location = Path(filename)
    
    zipdir = location.parent / location.stem
    with ZipFile(location) as myzip:
        myzip.extractall(zipdir)
    return zipdir

zipdir = unzip(location)
zipdir

PosixPath('data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f')

In [43]:
def get_unzipped_files(zipdir):
    filedir = zipdir / 'files'
    filenames = os.listdir(filedir)
    return [filedir / f for f in filenames]

file_paths = get_unzipped_files(zipdir)
pprint(file_paths)

[PosixPath('data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_udm2_clip.tif'),
 PosixPath('data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_AnalyticMS_SR_clip_bandmath.tif'),
 PosixPath('data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_AnalyticMS_metadata_clip.xml'),
 PosixPath('data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_AnalyticMS_DN_udm_clip.tif'),
 PosixPath('data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_metadata.json')]


##### 5.2: Visualize Images

In this section we will find the image files and their associated UDMs and we will visualize them.

In [45]:
def get_image_and_udm_files(file_paths):
    files = [str(p) for p in file_paths]
    
    # the image files are tiffs and are identified with '_SR_' in the name
    img_id = '_AnalyticMS_SR_'
    imgfiles = [f for f in files
                if f.endswith('.tif') and img_id in f]
    
    # get associated udm files for image files
    # each image has a unique id at the beginning of the name
    imgroots = [str(f).split(img_id)[0] for f in imgfiles]
    
    # the udm files are identified with '_udm2' in the name
    udmfiles = [next(f for f in files if f.startswith(r + '_udm2'))
                for r in imgroots]
    
    return imgfiles, udmfiles

imgfiles, udmfiles = get_image_and_udm_files(file_paths)
pprint(imgfiles)
pprint(udmfiles)

['data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_AnalyticMS_SR_clip_bandmath.tif']
['data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_udm2_clip.tif']


In [46]:
udmfile = udmfiles[0]
imgfile = imgfiles[0]

In [47]:
!gdalinfo $udmfile

Driver: GTiff/GeoTIFF
Files: data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_udm2_clip.tif
Size is 4632, 1946
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 15N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 15N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-93,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
        PARAMETER["False 

In [48]:
!gdalinfo $imgfile

Driver: GTiff/GeoTIFF
Files: data/demo/tutorial_order_9084691e-93ad-4eb0-9746-8ac14ab0e54f/files/20190415_170304_85_1068_3B_AnalyticMS_SR_clip_bandmath.tif
Size is 4633, 1946
Coordinate System is:
PROJCRS["WGS 84 / UTM zone 15N",
    BASEGEOGCRS["WGS 84",
        DATUM["World Geodetic System 1984",
            ELLIPSOID["WGS 84",6378137,298.257223563,
                LENGTHUNIT["metre",1]]],
        PRIMEM["Greenwich",0,
            ANGLEUNIT["degree",0.0174532925199433]],
        ID["EPSG",4326]],
    CONVERSION["UTM zone 15N",
        METHOD["Transverse Mercator",
            ID["EPSG",9807]],
        PARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8801]],
        PARAMETER["Longitude of natural origin",-93,
            ANGLEUNIT["degree",0.0174532925199433],
            ID["EPSG",8802]],
        PARAMETER["Scale factor at natural origin",0.9996,
            SCALEUNIT["unity",1],
            ID["EPSG",8805]],
       